In [167]:
import pandas as pd
import openai
import numpy as np
from tqdm.notebook import tqdm
import pickle
from transformers import GPT2TokenizerFast

ENGINE_NAME = "curie"

DOC_EMBEDDING_MODEL = f"text-search-{ENGINE_NAME}-doc-001"
QUERY_EMBEDDING_MODEL = f"text-search-{MODEL_NAME}-query-001"

COMPLETIONS_MODEL = "text-davinci-002"

# Question Answering

Many use cases require GPT to respond to user questions with insightful answers. For example, a customer support chatbot may need to provide answers to common questions. The GPT models have picked up a lot of general knowledge in training, but we often need to ingest and use a body of more specific information.

In this notebook we will demonstrate a method for augmenting GPT with a large body of additional contextual information by using embeddings search and retrieval. We'll be using a dataset of Wikipedia articles about the 2020 Summer Olympic Games. Please see [this notebook](examples/fine-tuned_qa/olympics-1-collect-data.ipynb) to follow the data gathering process.

GPT-3 isn't an expert on the Olympics by default:

In [176]:
prompt = "Who won the 2020 Summer Olympics men's high jump?"

openai.Completion.create(
    prompt=prompt,
    temperature=0,
    max_tokens=300,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    engine=COMPLETIONS_MODEL
)["choices"][0]["text"].strip("\n")

"The 2020 Summer Olympics men's high jump was won by Mariusz Przybylski of Poland."

Mariusz Przybylski is a professional footballer from Poland, and not much of a high jumper! Evidently GPT-3 needs some assistance here. (In fact we'd ideally like the model to be more conservative and say "I don't know" rather than making a guess - see [this guide](examples/fine-tuned_qa) for an exploration of that topic.)

When the total required context is short, we can include it in the prompt directly. In this case, we can use this information taken from Wikipedia:

In [179]:
prompt = """
The men's high jump event at the 2020 Summer Olympics took place between 30 July and 1 August 2021 at the Olympic Stadium.
33 athletes from 24 nations competed; the total possible number depended on how many nations would use universality places 
to enter athletes in addition to the 32 qualifying through mark or ranking (no universality places were used in 2021).
Italian athlete Gianmarco Tamberi along with Qatari athlete Mutaz Essa Barshim emerged as joint winners of the event following
a tie between both of them as they cleared 2.37m. Both Tamberi and Barshim agreed to share the gold medal in a rare instance
where the athletes of different nations had agreed to share the same medal in the history of Olympics. 
Barshim in particular was heard to ask a competition official "Can we have two golds?" in response to being offered a 
'jump off'. Maksim Nedasekau of Belarus took bronze. The medals were the first ever in the men's high jump for Italy and 
Belarus, the first gold in the men's high jump for Italy and Qatar, and the third consecutive medal in the men's high jump
for Qatar (all by Barshim). Barshim became only the second man to earn three medals in high jump, joining Patrik Sjöberg
of Sweden (1984 to 1992).

Who won the 2020 Summer Olympics men's high jump?"""

openai.Completion.create(
    prompt=prompt,
    temperature=0,
    max_tokens=300,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    engine=COMPLETIONS_MODEL
)["choices"][0]["text"].strip("\n")

"Gianmarco Tamberi and Mutaz Essa Barshim won the 2020 Summer Olympics men's high jump."

But this technique only works when the dataset of extra content that the model may need to know is small enough to fit in a single prompt. What do we do when we need the model to choose relevant contextual information from within a large body of information?

**In this notebook we demonstrate a method for augmenting GPT with a large body of additional contextual information by using embeddings search and retrieval.** This method answers queries in two steps: first it retrieves the information relevant to the query, then it writes an answer tailored to the question based on the retrieved information. The first step uses the Embedding API, the second step uses the Completion API.
 
The steps are:
* Preprocess the contextual information by splitting it into chunks and create an embedding vector for each chunk.
* On receiving a query, embed the query in the same vector space as the context chunks and find the context embeddings which are most similar to the query.
* Prepend the most relevant context embeddings to the query prompt.
* Submit the question along with the most relevant context to GPT, and receive an answer which makes use of the provided contextual information.

# 1) Preprocess the contextual information

We preprocess the contextual information by creating an embedding vector for each chunk of context in the knowledge base. An embedding is a vector of numbers that helps us understand how similar or different things are. The closer two embeddings are to each other, the more similar the things are that they represent.

This indexing stage can be executed offline and only runs once to precompute the indexes for the dataset so that each piece of content can be retrieved later. Since this is a small example, we will store and search the embeddings locally. If you have a larger dataset, consider using a vector search engine like Pinecone or Weaviate to power the search.

For the purposes of this tutorial we chose to use Curie embeddings, which are at a very good price and performance point. Since we will be using these embeddings for retrieval, we’ll use the search embeddings. 

In [22]:
# Load the dataset, obtained from this notebook <LINK>.
# This dataset has already been split into sections, one row for each section of the Wikipedia page.

df = pd.read_csv('olympics_sections.csv')
df = df.set_index(["title", "heading"])
print(f"{len(df)} rows in the data.")
df.sample(5)

3980 rows in the data.


,,content,tokens
title,heading,,
United States at the 2020 Summer Olympics,Diving,U.S. divers qualified for the following indivi...,89
Austria at the 2020 Summer Olympics,Summary,Austria competed at the 2020 Summer Olympics i...,115
2020 Women's Rugby Sevens Final Olympic Qualification Tournament,Knockout stage,"With two Olympic places available, the top eig...",49
Italy at the 2020 Summer Olympics,Karate,Italy entered five karateka into the inaugural...,148
2020 United States Olympic Team Trials (wrestling),Summary,The 2020 United States Olympic Team Trials for...,119


In [215]:
def get_embedding(text: str, engine: str) -> np.array:
    result = openai.Engine(engine).embeddings(input=text)
    return np.array(result["data"][0]["embedding"])

def get_doc_embedding(text: str) -> np.array:
    return get_embedding(text, DOC_EMBEDDING_MODEL)

def get_query_embedding(text: str) -> np.array:
    return get_embedding(text, QUERY_EMBEDDING_MODEL)

def compute_doc_embeddings(df: pd.DataFrame) -> pd.DataFrame:
    """
    Create an embedding for each document in the dataframe using the OpenAI Embeddings API.
    
    Return a dictionary that maps between each embedding vector and the index of the row that it corresponds to.
    """
    return {
        idx: get_doc_embedding(r.content) for idx, r in tqdm(df.iterrows(), total=len(df))
    }

In [48]:
%%time
context_embeddings = compute_doc_embeddings(df)

  0%|          | 0/3980 [00:00<?, ?it/s]

CPU times: user 1min 46s, sys: 7.92 s, total: 1min 54s
Wall time: 11min 16s


In [249]:
import sys
sys.getsizeof(context_embeddings) / 1000

147.552

# 2) Find the most similar context embeddings to the question embedding

At the time of question-answering, to answer the user's query we compute the embedding of the question and use it to find the most similar context chunks.


In [210]:
def vector_similarity(x: np.array, y: np.array) -> float:
    return np.dot(x, y)

def order_contexts_by_query_similarity(query: str, contexts: dict[(str, str), np.array]) -> list[(float, (str, str))]:
    """
    Find the query embedding for the supplied query, and compare it against all of the pre-calculated context embeddings
    to find the most relevant contexts. 
    
    Return the list of contexts, sorted by relevance in descending order.
    """
    query_embedding = get_query_embedding(query)
    
    context_similarities = sorted([
        (vector_similarity(query_embedding, doc_embedding), doc_index) for doc_index, doc_embedding in contexts.items()
    ], reverse=True)
    
    return context_similarities

In [211]:
order_contexts_by_query_similarity("Who won the men's high jump?", context_embeddings)[:5]

[(0.4296262686167575,
  ("Athletics at the 2020 Summer Olympics – Men's high jump", 'Summary')),
 (0.4067051275316437,
  ("Athletics at the 2020 Summer Olympics – Women's high jump", 'Summary')),
 (0.4046927788902179,
  ("Athletics at the 2020 Summer Olympics – Men's high jump", 'Background')),
 (0.40424431005550354,
  ("Athletics at the 2020 Summer Olympics – Men's triple jump", 'Summary')),
 (0.4021923762547752,
  ("Athletics at the 2020 Summer Olympics – Women's long jump", 'Summary'))]

We can see that summaries information about the men's and women's high jump competitions are judged to be the most relevant contexts for this question, which is in line with what we would expect.

# 3) Add the most relevant contexts to the query prompt

Once we've calculated the most relevant pieces of context, we construct a prompt by simply prepending them to the supplied query. It is helpful to use a query separator to help the model distinguish between separate pieces of text.

In [214]:
MAX_CONTEXT_LEN = 500
CONTEXT_SEPARATOR = "\n* "

tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
context_separator_len = len(tokenizer.tokenize(CONTEXT_SEPARATOR))

f"Context separator contains {context_separator_len} tokens"

'Context separator contains 3 tokens'

In [160]:
def construct_prompt(question: str, contexts: dict, df: pd.DataFrame) -> str:
    """
    
    """
    most_relevant_contexts = order_contexts_by_query_similarity(question, doc_embeddings)
    
    chosen_contexts = []
    chosen_contexts_indexes = []
    chosen_contexts_len = 0
     
    for _, context_index in most_relevant_contexts:
        # Add contexts until we run out of space.
        # In this version, we will not add the final context that overflows our limit; 
        # you may wish to "partially add" that final context.
        
        context = df.loc[context_index]
        
        chosen_contexts_len += context.tokens + context_separator_len
        if chosen_contexts_len > MAX_CONTEXT_LEN:
            break
            
        chosen_contexts.append(CONTEXT_SEPARATOR + context.content)
        chosen_contexts_indexes.append(str(context_index))
        
    print(f"Selected {len(chosen_contexts)} contexts:")
    print("\n".join(chosen_contexts_indexes))
        
    return "".join(chosen_contexts) + "\n\n" + question + "\n"

In [161]:
prompt = construct_prompt(
    "Who won the 2020 Summer Olympics men's high jump?",
    doc_embeddings,
    df
)

print("===\n", prompt)

Selected 3 contexts:
("Athletics at the 2020 Summer Olympics – Women's high jump", 'Summary')
("Athletics at the 2020 Summer Olympics – Men's high jump", 'Summary')
("Athletics at the 2020 Summer Olympics – Men's triple jump", 'Summary')
===
 
* The women's high jump event at the 2020 Summer Olympics took place on 5 and 7 August 2021 at the Japan National Stadium. Even though 32 athletes qualified through the qualification system for the Games, only 31 took part in the competition. This was the 22nd appearance of the event, having appeared at every Olympics since women's athletics was introduced in 1928.
* The men's high jump event at the 2020 Summer Olympics took place between 30 July and 1 August 2021 at the Olympic Stadium. 33 athletes from 24 nations competed; the total possible number depended on how many nations would use universality places to enter athletes in addition to the 32 qualifying through mark or ranking (no universality places were used in 2021). Italian athlete Gianm

# 4) Submit the extended query to the Completions API.

Now that we've retrieved the relevant context, we can finally use the Completions API to answer the user's query.

In [233]:
COMPLETIONS_API_PARAMS = {
    "temperature": 0.0,
    "max_tokens": 300,
    "top_p": 1,
    "frequency_penalty": 0,
    "presence_penalty": 0,
    "engine": COMPLETIONS_MODEL,
    "stop": "\n\n"
}

In [234]:
def answer_query_with_context(
    query: str,
    df: pd.DataFrame,
    context_embeddings: dict[(str, str), np.array]
) -> str:
    prompt = construct_prompt(
        query,
        context_embeddings,
        df
    )

    response = openai.Completion.create(
                prompt=prompt,
                **COMPLETIONS_API_PARAMS
            )

    return response["choices"][0]["text"].strip("\n")

In [235]:
answer_query_with_context("Who won the 2020 Summer Olympics men's high jump?", df, context_embeddings)

Selected 3 contexts:
("Athletics at the 2020 Summer Olympics – Women's high jump", 'Summary')
("Athletics at the 2020 Summer Olympics – Men's high jump", 'Summary')
("Athletics at the 2020 Summer Olympics – Men's triple jump", 'Summary')


"Gianmarco Tamberi and Mutaz Essa Barshim both cleared 2.37m to win the gold medal in the men's high jump event at the 2020 Summer Olympics."

# More Examples

In [242]:
query = "Why was the 2020 Summer Olympics originally postponed?"
answer_query_with_context(query, df, context_embeddings)

Selected 1 contexts:
('2020 Summer Olympics', 'Postponement to 2021')


'The 2020 Summer Olympics were originally postponed due to the COVID-19 pandemic. The outbreak of the pandemic began in China in December 2019, and spread to Japan in January 2020. The outbreak was declared a Public Health Emergency of International Concern by the World Health Organization on 30 January 2020. On 25 February 2020, the IOC announced that it would hold a meeting on the following day to discuss the outbreak and its potential impact on the Games.'

In [236]:
query = "In the 2020 Summer Olympics, how many gold medals did the country which won the most medals win?"
answer_query_with_context(query, df, context_embeddings)

Selected 1 contexts:
('2020 Summer Olympics medal table', 'Summary')


'The United States won the most medals overall, with 113, and the most gold medals, with 39.'

In [238]:
query = "What was unusual about the men’s shotput competition?"
answer_query_with_context(query, df, context_embeddings)

Selected 3 contexts:
("Athletics at the 2020 Summer Olympics – Men's shot put", 'Summary')
("Athletics at the 2020 Summer Olympics – Men's shot put", 'Background')
("Athletics at the 2020 Summer Olympics – Men's hammer throw", 'Competition format')


'The men’s shotput competition at the 2020 Summer Olympics was unusual because the same three competitors received the same medals in back-to-back editions of the same individual event. Americans Ryan Crouser and Joe Kovacs and New Zealander Tom Walsh repeated their gold, silver, and bronze (respectively) performances from the 2016 Summer Olympics. They became the 15th, 16th, and 17th men to earn multiple medals in the shot put; Crouser was the 4th to repeat as champion.'

In [239]:
query = "In the 2020 Summer Olympics, how many silver medals did Italy win?"
answer_query_with_context(query, df, context_embeddings)

Selected 1 contexts:
('Italy at the 2020 Summer Olympics', 'Summary')


'10'